In [42]:
import torch
print(torch.__version__)
torch.cuda.empty_cache()

2.0.0+cu118


In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)



cuda


In [82]:
import textattack
import transformers
from textattack import Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.constraints.pre_transformation.max_modification_rate import MaxModificationRate
dataset = HuggingFaceDataset("imdb", split="test")
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder

from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
    MaxModificationRate
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import BERT
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapMaskedLM


def build_A2T(model_wrapper, mlm=False):
    constraints = [RepeatModification(), StopwordModification()]
    input_column_modification = InputColumnModification(
        ["premise", "hypothesis"], {"premise"}
    )
    constraints.append(input_column_modification)
    constraints.append(PartOfSpeech(allow_verb_noun_swap=False))
    constraints.append(MaxModificationRate(max_rate=0.1, min_threshold=4))
    sent_encoder = BERT(
        model_name="stsb-distilbert-base", threshold=0.9, metric="cosine"
    )
    constraints.append(sent_encoder)

    if mlm:
        transformation = transformation = WordSwapMaskedLM(
            method="bae", max_candidates=20, min_confidence=0.0, batch_size=16
        )
    else:
        transformation = WordSwapEmbedding(max_candidates=20)
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.8))

    #
    # Goal is untargeted classification
    #
    goal_function = UntargetedClassification(model_wrapper, target_max_score=0.8, model_batch_size=32)
    #
    # Greedily swap words with "Word Importance Ranking".
    #
    search_method = GreedyWordSwapWIR(wir_method="gradient")

    return Attack(goal_function, constraints, transformation, search_method)


def build_textFooler(model_wrapper):
    #
    # Swap words with their 50 closest embedding nearest-neighbors.
    # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
    #
    transformation = WordSwapEmbedding(max_candidates=50)
    #
    # Don't modify the same word twice or the stopwords defined
    # in the TextFooler public implementation.
    #
    # fmt: off
    stopwords = set(
        ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", "almost", "alone",
         "along", "already", "also", "although", "am", "among", "amongst", "an", "and", "another", "any", "anyhow",
         "anyone", "anything", "anyway", "anywhere", "are", "aren", "aren't", "around", "as", "at", "back", "been",
         "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "both", "but",
         "by", "can", "cannot", "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don",
         "don't", "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", "everyone",
         "everything", "everywhere", "except", "first", "for", "former", "formerly", "from", "hadn", "hadn't", "hasn",
         "hasn't", "haven", "haven't", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon",
         "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "i", "if", "in", "indeed", "into",
         "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me",
         "meanwhile", "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't",
         "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no", "nobody",
         "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on", "once", "one", "only",
         "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "per", "please", "s",
         "same", "shan", "shan't", "she", "she's", "should've", "shouldn", "shouldn't", "somehow", "something",
         "sometime", "somewhere", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them",
         "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these",
         "they", "this", "those", "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under",
         "unless", "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
         "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein",
         "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose",
         "why", "with", "within", "without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "you'd",
         "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]
    )
    # fmt: on
    constraints = [RepeatModification()
                   # StopwordModification(stopwords=stopwords)
        , MaxModificationRate(max_rate=0.5, min_threshold= 10)]
    #
    # During entailment, we should only edit the hypothesis - keep the premise
    # the same.
    #
    input_column_modification = InputColumnModification(
        ["premise", "hypothesis"], {"premise"}
    )
    constraints.append(input_column_modification)
    # Minimum word embedding cosine similarity of 0.5.
    # (The paper claims 0.7, but analysis of the released code and some empirical
    # results show that it's 0.5.)
    #
    constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
    #
    # Only replace words with the same part of speech (or nouns with verbs)
    #
    constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
    #
    # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
    #
    # In the TextFooler code, they forget to divide the angle between the two
    # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
    # new threshold is 1 - (0.5) / pi = 0.840845057
    #
    use_constraint = UniversalSentenceEncoder(
        threshold=0.840845057,
        metric="angular",
        compare_against_original=False,
        window_size=15,
        skip_text_shorter_than_window=True,
    )
    constraints.append(use_constraint)
    #
    # Goal is untargeted classification
    #
    goal_function = UntargetedClassification(model_wrapper, target_max_score=0.7)
    #
    # Greedily swap words with "Word Importance Ranking".
    #
    search_method = GreedyWordSwapWIR(wir_method="delete")

    return Attack(goal_function, constraints, transformation, search_method)


# Load target model, tokenizer, and model_wrapper
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb").to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

attack_type = 'combined'
attack_args = textattack.AttackArgs(
    num_examples=100,
    log_to_csv=f"log_{attack_type}.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False
)
dataset = HuggingFaceDataset("imdb", split="test")

# Now, let's make the attack from the 4 components:
attack = build_textFooler(model_wrapper)
# attack = TextFoolerJin2019.build(model_wrapper)
# attack = Attack(goal_function, constraints, transformation, search_method)
attacker = Attacker(attack, dataset, attack_args)
res = attacker.attack_dataset()

print('============================')
print(res)

Reusing dataset imdb (C:\Users\zeyu song\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


textattack: Loading datasets dataset imdb, split test.


Reusing dataset imdb (C:\Users\zeyu song\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


textattack: Loading datasets dataset imdb, split test.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log_combined.csv

  0%|          | 0/100 [00:00<?, ?it/s]


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): MaxModificationRate(
        (max_rate):  0.5
        (min_threshold):  10
      )
    (5): InputColumnModification(
     

...

--------------------------------------------- Result 99 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (100%)]]

Like 'Singin' in the Rain', 'Cover Girl' has a trio of two guys and a girl. In 'Cover Girl', Phil Silvers (Genius) is the comic relief. He corresponds to Donald O'Connor's funny man part in 'Singin in the Rain'. In Cover Girl, Gene Kelly's love interest is Rita Hayworth and in 'Rain', it's Debbie Reynolds. That's where the comparison ends.<br /><br />Whereas "Singin' in the Rain' is a classic American movie musical, 'Cover Girl' is mediocrity incarnate. The story isn't very complicated. Rusty Parker (Rita Hayworth) is a dancer in Danny MacGuire's low-rent nightclub in Brooklyn. Rusty decides to enter a Cover Girl contest sponsored by a wealthy publisher, John Coudair, who made an unsuccessful play for Rusty's grandmother years ago. Coudair introduces Rusty to Broadway producer Noel Wheaton who makes her into a star. Danny feels slighted when


[Succeeded / Failed / Skipped / Total] 90 / 1 / 8 / 99: 100%|██████████| 100/100 [19:21<00:00, 11.61s/it]
[Succeeded / Failed / Skipped / Total] 91 / 1 / 8 / 100: 100%|██████████| 100/100 [19:21<00:00, 11.61s/it]textattack: Saving checkpoint under "checkpoints\1682477178124.ta.chkpt" at 2023-04-25 22:46:18 after 100 attacks.


--------------------------------------------- Result 100 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (98%)]]

Rita Hayworth plays a Brooklyn nightclub dancer named Rusty who specializes in cheesecake chorus revues; she manages to get herself on the cover of a national fashion magazine, but her impending success as a solo (with romantic offers all around) has smitten boss Gene Kelly [[chomping]] at the bit. [[Terribly]] tired [[piece]] [[of]] Technicolor cotton candy, with unmemorable musical sketches (the two worst of which are irrelevant flashbacks to the 1890s, with [[Hayworth]] portraying her own grandmother). Kelly, as always, [[dances]] well but acts with false sincerity; when he's serious, he's insufferable, and the rest of the time he's flying on adrenaline. The script is a lead weight, [[not]] even giving supporting players Phil Silvers and Eve Arden any good lines. *1/2 from ****

Rita Hayworth plays a Brooklyn nightclub dancer named Rusty 

[Succeeded / Failed / Skipped / Total] 91 / 1 / 8 / 100: 100%|██████████| 100/100 [19:21<00:00, 11.61s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 91     |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 8      |
| Original accuracy:            | 92.0%  |
| Accuracy under attack:        | 1.0%   |
| Attack success rate:          | 98.91% |
| Average perturbed word %:     | 8.38%  |
| Average num. words per input: | 244.84 |
| Avg num queries:              | 692.5  |
+-------------------------------+--------+
[<textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x000001CA39612B20>, <textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x000001CA03235880>, <textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x000001CADF41BD30>, <textattack.attack_results.successful_attack_result.SuccessfulAttackResult object at 0x000001CA38BC7160>, <textattack.att

In [88]:
import pandas as pd

df = pd.read_csv('attack-log/imdb/bert/textfooler_min_10/textfooler_min_10.csv')

In [77]:
df.loc[(df.result_type == 'Successful') &  (df.original_output == df.perturbed_output), 'result_type'] = 'Failed'

In [89]:
df

,Unnamed: 0,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,0,[[I]] [[love]] sci-fi and [[am]] [[willing]] t...,[[me]] [[enjoyed]] sci-fi and [[feel]] [[wish]...,0.000173,0.563655,0,1,0,1006,Successful
1,1,"Worth the entertainment value of a rental, esp...","Worth the entertainment value of a rental, esp...",0.003665,0.573658,0,1,0,246,Successful
2,2,its a totally average film with a few semi-alr...,its a totally average film with a few semi-alr...,0.003767,0.536536,0,1,0,154,Successful
3,3,[[STAR]] [[RATING]]: ***** Saturday Night ****...,[[STARLIGHT]] [[SYMBOL]]: ***** Saturday Night...,0.000160,0.540451,0,1,0,958,Successful
4,5,[[I]] had high [[hopes]] for this one until th...,[[me]] had high [[prospects]] for this one unt...,0.000208,0.903948,0,1,0,372,Successful
...,...,...,...,...,...,...,...,...,...,...
86,95,"This film [[seems]] well made, and more effort...","This film [[emerges]] well made, and more effo...",0.000218,0.929699,0,1,0,338,Successful
87,96,"It hurt to watch this movie, it really did... ...","It hurt to watch this movie, it really did... ...",0.000333,0.888271,0,1,0,186,Successful
88,97,Rita Hayworth is just stunning at [[times]] an...,Rita Hayworth is just stunning at [[length]] a...,0.000176,0.570742,0,1,0,437,Successful
89,98,"Like 'Singin' in the Rain', 'Cover Girl' has a...","Like 'Singin' in the Rain', 'Cover Girl' has a...",0.000291,0.998933,0,1,0,544,Successful


In [92]:
df.perturbed_text[0]

'[[me]] [[enjoyed]] sci-fi and [[feel]] [[wish]] to put [[upstairs]] [[into]] [[another]] lot. Sci-fi [[musicals]]/TV are usually underfunded, under-appreciated and misunderstood. I [[judged]] to like this, I [[indeed]] did, but it [[provides]] to [[nice]] TV sci-fi as Babylon 5 [[ai]] to Star Trek (the original). [[Punchy]] [[surgeon]], cheap cardboard sets, stilted dialogues, [[MC]] that doesn\'t match the background, [[nor]] [[terrifically]] one-dimensional characters [[pronounced]] be overcome [[of]] a \'sci-fi\' setting. (I\'m sure there are those of you [[outside]] there [[whoever]] think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) [[During]] US viewers might like emotion and character development, sci-fi is a [[sorting]] that does [[never]] [[assuming]] itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a

In [80]:
df.to_csv("attack-log/imdb/bert/textfooler/log_sub.csv")